In [1]:
import pandas as pd

# Load the csv file into a DataFrame
file_path = '../Datasets/SPX.csv'
data = pd.read_csv(file_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)

# Display the first few rows of the DataFrame
data.head()


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,1845.859985,1845.859985,1827.739990,1831.979980,1831.979980,3080600000
2014-01-03,1833.209961,1838.239990,1829.130005,1831.369995,1831.369995,2774270000
2014-01-06,1832.310059,1837.160034,1823.729980,1826.770020,1826.770020,3294850000
2014-01-07,1828.709961,1840.099976,1828.709961,1837.880005,1837.880005,3511750000
2014-01-08,1837.900024,1840.020020,1831.400024,1837.489990,1837.489990,3652140000


In [2]:
# # Calculate Simple Moving Averages (SMA)
# spx_data['SMA_50'] = spx_data['Close'].rolling(window=50).mean()
# spx_data['SMA_200'] = spx_data['Close'].rolling(window=200).mean()

# # Calculate Relative Strength Index (RSI)
# delta = spx_data['Close'].diff()
# gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
# loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
# rs = gain / loss
# spx_data['RSI_14'] = 100 - (100 / (1 + rs))

# # Calculate Moving Average Convergence Divergence (MACD)
# ema_12 = spx_data['Close'].ewm(span=12, adjust=False).mean()
# ema_26 = spx_data['Close'].ewm(span=26, adjust=False).mean()
# spx_data['MACD'] = ema_12 - ema_26
# spx_data['MACD_Signal'] = spx_data['MACD'].ewm(span=9, adjust=False).mean()
# spx_data['MACD_Hist'] = spx_data['MACD'] - spx_data['MACD_Signal']

# # Display the first few rows with the new indicators
# spx_data[['Close', 'SMA_50', 'SMA_200', 'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist']].head()


In [3]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()
    

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            

C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
from backtesting import Backtest

bt = Backtest(data, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats


Start                     2014-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                    98.72814
Equity Final [$]                  6205.197739
Equity Peak [$]                  10177.311075
Return [%]                         -37.948023
Buy & Hold Return [%]              160.364749
Return (Ann.) [%]                   -4.667142
Volatility (Ann.) [%]               11.925297
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -48.378846
Avg. Drawdown [%]                   -9.975921
Max. Drawdown Duration     3584 days 00:00:00
Avg. Drawdown Duration      720 days 00:00:00
# Trades                                  119
Win Rate [%]                        32.773109
Best Trade [%]                      13.493278
Worst Trade [%]                    -13.630685
Avg. Trade [%]                    

In [6]:
bt.plot()


C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1512', ...)

In [7]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats


CPU times: total: 2.84 s
Wall time: 2.84 s


Start                     2014-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   98.370429
Equity Final [$]                 11944.584387
Equity Peak [$]                   13170.55906
Return [%]                          19.445844
Buy & Hold Return [%]              160.364749
Return (Ann.) [%]                    1.795686
Volatility (Ann.) [%]               14.806846
Sharpe Ratio                         0.121274
Sortino Ratio                         0.17815
Calmar Ratio                         0.089486
Max. Drawdown [%]                  -20.066711
Avg. Drawdown [%]                   -3.833179
Max. Drawdown Duration     1275 days 00:00:00
Avg. Drawdown Duration       91 days 00:00:00
# Trades                                   73
Win Rate [%]                        39.726027
Best Trade [%]                      19.346371
Worst Trade [%]                     -7.865184
Avg. Trade [%]                    

In [8]:
stats._strategy


<Strategy SmaCross(n1=20,n2=35)>

In [9]:
bt.plot(plot_volume=False, plot_pl=False)


C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2465', ...)

In [10]:
stats.tail()


Expectancy [%]                                             0.340045
SQN                                                        0.550481
_strategy                                     SmaCross(n1=20,n2=35)
_equity_curve                       Equity  DrawdownPct Drawdown...
_trades               Size  EntryBar  ExitBar   EntryPrice    Ex...
dtype: object

In [11]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.


,Equity,DrawdownPct,DrawdownDuration
2014-01-02,10000.000000,0.000000,NaT
2014-01-03,10000.000000,0.000000,NaT
2014-01-06,10000.000000,0.000000,NaT
2014-01-07,10000.000000,0.000000,NaT
2014-01-08,10000.000000,0.000000,NaT
...,...,...,...
2023-12-22,11888.084387,0.097374,NaT
2023-12-26,11928.324621,0.094319,NaT
2023-12-27,11941.984777,0.093282,NaT
2023-12-28,11945.524817,0.093013,NaT


In [12]:
stats['_trades']  # Contains individual trade data


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,41,71,1852.928440,1831.449951,-107.392445,-0.011592,2014-03-04,2014-04-15,42 days
1,-5,71,79,1827.787051,1865.000000,-186.064745,-0.020360,2014-04-15,2014-04-28,13 days
2,5,79,83,1868.730000,1885.300049,82.850245,0.008867,2014-04-28,2014-05-02,4 days
3,-5,83,87,1881.529449,1877.390015,20.697170,0.002200,2014-05-02,2014-05-08,6 days
4,5,87,88,1881.144795,1875.270020,-29.373875,-0.003123,2014-05-08,2014-05-09,1 days
...,...,...,...,...,...,...,...,...,...,...
68,2,2333,2426,4118.510619,4396.439941,555.858644,0.067483,2023-04-11,2023-08-23,134 days
69,-2,2426,2446,4387.647061,4374.359863,26.574396,0.003028,2023-08-23,2023-09-21,29 days
70,2,2446,2452,4383.108583,4328.180176,-109.856813,-0.012532,2023-09-21,2023-09-29,8 days
71,-2,2452,2488,4319.523816,4511.700195,-384.352759,-0.044490,2023-09-29,2023-11-20,52 days
